In [1]:
import codecs, json
from os import listdir
from os.path import isfile, join
import pymongo
import os

In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
import spacy
nlp = spacy.load("es_core_news_md")

In [3]:
import pickle

In [16]:
client = pymongo.MongoClient("mongodb://54.90.49.105:27017", username='rw_admin_smdt', password='HolocaustoDel84', authSource="smdt")
db = client["smdt"]
queries_collection = db['queries']
tweets_collection = db['tweets']
stats_collection = db['col_stats']

In [5]:
corpus_file = open('../taxonomia/corpus_sexperto_with_lemma_with_s-08062020.pkl', 'rb')
corpus = pickle.load(corpus_file)
corpus_list = [j for sub in corpus for j in sub]
corpus_set = set(corpus_list)

In [6]:
corpus_set

{'hombrestransexuales',
 'tipicas',
 'cuidadosamente',
 'objetivar',
 'alargador',
 'cicatrizar',
 'continuamente',
 'relajacion',
 'latransmisionde',
 'elliquido',
 'costumbre',
 'prescribir',
 'ponte',
 'desconcentracion',
 'prematuro',
 'aneyaculacion',
 'colposcopio',
 'medula',
 'dietario',
 'despertador',
 'actor',
 'safari',
 'envia',
 'inyeccion',
 'pera',
 'alargamiento',
 'trimestre',
 'neurologicos',
 'encasillarse',
 'laparoscopia',
 'tornar',
 'colocarte',
 'polisopreno',
 'condicion',
 'responsable',
 'extirpacion',
 'pronunciamiento',
 'granito',
 'ensancharme',
 'ciclovias',
 'oler',
 'percibir',
 'union',
 'practicar',
 'final',
 'protesis',
 'sentirme',
 'pnsdsdr',
 'barrera',
 'componente',
 'contarnos',
 'equipar',
 'progesterona',
 'digital',
 'prep',
 'abusador',
 'contagioso',
 'jaquear',
 'participante',
 'vaginitis',
 'especialista',
 'gordo',
 'anorquia',
 'cafe',
 'autoridad',
 'menarquia',
 'penetrativo',
 'negativo',
 'despenalizado',
 'liquen',
 'huevar',


In [7]:
# function to clean text
def tweet_to_vectors(raw_review):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 

    # 2. Convert to lower case and only words
    only_words = letters_only.lower()
    only_words = " ".join(only_words.split())
    # 2.1 the nlp doc and words ❤️ 
    doc = nlp(only_words)
    lexical_tokens = [tok.lemma_ for tok in doc if tok.pos_ != 'PRON' and not (tok.is_punct | tok.is_stop)]
    words = [t.lower() for t in lexical_tokens if len(t) > 2 and t.isalpha()]
    # 3. Remove Stopwords. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("spanish"))  
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  #returns a list 
    # 5. Stem words. Need to define porter stemmer above
    meaningful_words = [(lambda x: stemmer.stem(x) if len(x) > 5 else x)(word) for word in meaningful_words]
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( meaningful_words ) 

In [8]:
# function to clean text
def tweet_to_lemmas(raw_review):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 

    # 2. Convert to lower case and only words
    only_words = letters_only.lower()
    only_words = " ".join(only_words.split())
    # 2.1 the nlp doc and words ❤️ 
    doc = nlp(only_words)
    lexical_tokens = [tok.lemma_ for tok in doc if tok.pos_ != 'PRON' and not (tok.is_punct | tok.is_stop)]
    words = [t.lower() for t in lexical_tokens if len(t) > 2 and t.isalpha()]
    # 3. Remove Stopwords. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("spanish"))  
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  #returns a list 
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( meaningful_words ) 

In [9]:
def similarity_calculator(c, t):
    i = c.intersection(t)
    return float(len(i)) / (len(c) + len(t) - len(i))

In [11]:
for tweet in tweets_collection.find({"text" : "Fue embarazo no planeado, la estúpida pensaba en abortar, pero ahora ya, ya es una bendición"})[:1]:
    if 'text' in tweet:
        tweet['stemmas'] = tweet_to_vectors(tweet['text'])
        tweet['lemmas'] = tweet_to_lemmas(tweet['text'])
        tweet_set = set(tweet['stemmas'])
        #tweet['similarity'] = similarity_calculator(corpus_set, tweet_set)
    print(tweet)

{'_id': ObjectId('5ef50951054b870f007cc9a1'), 'has_media': False, 'hashtags': [], 'img_urls': [], 'is_replied': False, 'is_reply_to': False, 'likes': 0, 'links': [], 'parent_tweet_id': '', 'replies': 0, 'reply_to_users': [], 'retweets': 0, 'screen_name': 'noel_exe', 'text': 'Fue embarazo no planeado, la estúpida pensaba en abortar, pero ahora ya, ya es una bendición', 'timestamp': '2014-08-31T18:06:55', 'timestamp_epochs': 1409508415, 'tweet_id': '506141096154787841', 'tweet_url': '/noel_exe/status/506141096154787841', 'user_id': '863509381', 'username': 'NOEL2000', 'video_url': '', 'queryName': 'embarazo-AND-planeado', 'query': ['embarazo', 'planeado'], 'stemmas': ['embaraz', 'plan', 'est', 'pedir', 'pens', 'abort', 'bendici'], 'lemmas': ['embarazar', 'planear', 'est', 'pedir', 'pensar', 'abortar', 'bendici'], 'similarity': 0.0008079719903043361, 'q_versions': [2]}


In [31]:
bulkk = pymongo.bulk.BulkOperationBuilder(tweets_collection,ordered=False)
for tweet in tweets_collection.find():
    if 'text' in tweet:
        #tweet['stemmas'] = tweet_to_vectors(tweet['text'])
        #tweet['lemmas'] = tweet_to_lemmas(tweet['text'])
        tweet_set = set(tweet['stemmas'])
        #tweet['similarity'] = similarity_calculator(corpus_set, tweet_set)
        try:
            #tweets_collection.find_one_and_update(filter={'tweet_id': tweet['tweet_id']}, update={'$set': tweet})
            bulkk.find( { '_id': tweet['_id']}).update({ '$set': {'similarity': similarity_calculator(corpus_set, tweet_set)}})
            #print('{}. '.format(ccc), tweet['tweet_id'], tweet['similarity'])
        except Exception as be:
            print("error on update tweet", be)
            continue

In [32]:
bulkk.execute()

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 398936,
 'nModified': 366090,
 'nRemoved': 0,
 'upserted': []}

In [10]:
paths_bk = []
paths = []
paths_bk = paths_bk+list(set(list(paths)))

In [79]:
paths_bk = ["{}-{}.json".format(q['name'],q['lastUpserted']) for q in queries_collection.find({})]

In [81]:
for pb in paths_bk:
    try:
        os.remove('queriesResults2/{}'.format(pb))
    except:
        a = 1
    try:
        os.remove('queriesResults/{}'.format(pb))
    except:
        a = 1

In [82]:
paths = [f for f in listdir('queriesResults2/') if isfile(join('queriesResults2/', f)) and ".json" in f and f not in paths_bk]
len(paths)

397

In [21]:
paths[0]

'sexo-AND-personar-1593137973805.json'

In [33]:
# get stats object
current_mean = 0
current_std = 0
current_var = 0
total_tweets = 0
tweets_stats = stats_collection.find_one()
if tweets_stats is not None:
    current_mean = tweets_stats['mean']
    current_std = tweets_stats['std']
    current_var = current_std ** 2
total_tweets = tweets_collection.count_documents({})

In [78]:
print(current_mean, current_std, current_var, total_tweets)

0.001596633375862045 0.0007770017300054408 6.041526587787017e-07 573031


In [38]:
#function to calculate new value of mean (N) given the previus mean (N-1) and new value (Xn) and length of set
def getNewMean(previus_mean, newvalue, n):
    return previus_mean + (newvalue - previus_mean)/(n + 1)

In [37]:
#function to calculate new value of standard deviation (N) given the previus mean (N-1) and new value (Xn)
#NOTE: send value of N as N+1 (new length of set, but not current)
def getNewVar(current_mean, current_var, new_value, new_mean, n):
    return ((n - 2) * current_var + (new_value - new_mean)*(new_value - current_mean))/(n-1)

In [43]:
from math import sqrt
getNewVar(4, 0, 100, 20, 6)

39.191835884530846

In [83]:
import sys
resultsTweetsInsertion = []
resultsQueriesInsertion = []
count_path = 0
bulk_results = None
for path in paths:
    path_name = "-".join(path.split(".")[0].split('-')[:-1])
    path_query = [q for q in path.split(".")[0].split('-')[:-1] if q != "AND"]
    millis = int("-".join(path.split(".")[0].split('-')[-1:]))
    count_path += 1
    try:
        query = queries_collection.find_one({"name": path_name})
        print(count_path, ". -- query", query, path_name)
        
        if query == None or (query["status"] == 1): 
            #if millis is greather than db.queries.lastUpserted
            with codecs.open('queriesResults2/{}'.format(path), 'r', 'utf-8') as file:
                tweets = json.load(file, encoding='utf-8')
                bulk = pymongo.bulk.BulkOperationBuilder(tweets_collection,ordered=False)
                for tweet in tweets:
                    tweet['queryName'] = path_name
                    tweet['query'] = path_query
                    if 'text' in tweet:
                        tweet['stemmas'] = tweet_to_vectors(tweet['text'])
                        tweet['lemmas'] = tweet_to_lemmas(tweet['text'])
                        tweet_set = set(tweet['stemmas'])
                        tweet['similarity'] = similarity_calculator(corpus_set, tweet_set)
                        
                    tweet['q_versions'] = [3]
                    if 'text_html' in tweet:
                        del tweet['text_html']
                    try:
                        bulk.insert(tweet)
                    except Exception as be:
                        print("error on bulk insertion", be)
                        continue
                    
                try:
                    bulk_results = bulk.execute()
                    if 'tweet_id' in tweets[0]:
                        print("tweet_id: ",tweet[0]["tweet_id"])
                except Exception as e:
                    print("duplicates inserting {} tweets".format(path_name), e)
                    #print('error', sys.exc_info())
                    #print('details', e.details)
                    error_details = e.details['writeErrors']
                
                # increment mean and sdt
                tweets_ids = [err["keyValue"]["tweet_id"] for err in error_details if "keyValue" in err and "tweet_id" in err["keyValue"]]
                for tweet in tweets:
                    if tweet['tweet_id'] not in tweets_ids:
                        new_mean = getNewMean(current_mean, tweet["similarity"], total_tweets)
                        total_tweets += 1
                        current_var = getNewVar(current_mean, current_var, tweet["similarity"], new_mean, total_tweets)
                        current_mean = new_mean
                try:
                    queries_collection.find_one_and_update(filter={'name': path_name},
                                                   update={'$set': {'status' : 1, 
                                                                    'lastUpserted': millis},
                                                           
                                                           '$setOnInsert': {'taxonomy': 'MAA', 
                                                                           'name': path_name, 
                                                                           'query': path_query,
                                                                           'q_versions': [3]} 
                                                          },
                                                   upsert=True)
                    try:
                        os.remove('queriesResults2/{}'.format(path))
                        print("file {} processed", path)

                    except Exception as fwde:
                        print("Failed file deletion", fwde)
                except Exception as ee:
                    print("fail inserting {} query".format(path_name), ee)
                
                    
                
        else:
            print("this query has been already upserted, skipping")
            try:
                os.remove('queriesResults2/{}'.format(path))
            except:
                a = 1
            try:
                os.remove('queriesResults/{}'.format(path))
            except:
                a = 1
    except Exception as es: 
        print("error", es)

1 . -- query None malo-AND-relacionar
duplicates inserting malo-AND-relacionar tweets batch op errors occurred
file {} processed malo-AND-relacionar-1594292520790.json
2 . -- query None recibir-AND-recomendar
duplicates inserting recibir-AND-recomendar tweets batch op errors occurred
file {} processed recibir-AND-recomendar-1594289926200.json
3 . -- query None compartir-AND-vida
duplicates inserting compartir-AND-vida tweets 0
error 'KeyError' object has no attribute 'details'
4 . -- query None hormonal-AND-metodos
duplicates inserting hormonal-AND-metodos tweets batch op errors occurred
file {} processed hormonal-AND-metodos-1594282647055.json
5 . -- query None asesoria-AND-medicar
duplicates inserting asesoria-AND-medicar tweets batch op errors occurred
file {} processed asesoria-AND-medicar-1594287730998.json
6 . -- query None disminuir-AND-hormona
duplicates inserting disminuir-AND-hormona tweets batch op errors occurred
file {} processed disminuir-AND-hormona-1594282511212.json
7 

duplicates inserting condon-AND-dispositivo tweets batch op errors occurred
file {} processed condon-AND-dispositivo-1594279004983.json
44 . -- query {'_id': ObjectId('5ef51956cfa8ed712cbb14b6'), 'name': 'anticonceptivo-AND-embarazar', 'lastUpserted': 1593113906600, 'q_versions': [2], 'query': ['anticonceptivo', 'embarazar'], 'status': 1, 'taxonomy': 'MAA'} anticonceptivo-AND-embarazar
duplicates inserting anticonceptivo-AND-embarazar tweets batch op errors occurred
file {} processed anticonceptivo-AND-embarazar-1594284147896.json
45 . -- query None barrera-AND-condon
duplicates inserting barrera-AND-condon tweets batch op errors occurred
file {} processed barrera-AND-condon-1594282809430.json
46 . -- query None ano-AND-página
duplicates inserting ano-AND-página tweets batch op errors occurred
file {} processed ano-AND-página-1594288334680.json
47 . -- query None haber-AND-preservativo
duplicates inserting haber-AND-preservativo tweets batch op errors occurred
file {} processed haber-A

duplicates inserting excitacion-AND-normal tweets batch op errors occurred
file {} processed excitacion-AND-normal-1594284875761.json
82 . -- query {'_id': ObjectId('5ef61a724060b4e3c8f2cff6'), 'name': 'causar-AND-contagiar', 'lastUpserted': 1593146935101, 'q_versions': [2], 'query': ['causar', 'contagiar'], 'status': 1, 'taxonomy': 'MAA'} causar-AND-contagiar
duplicates inserting causar-AND-contagiar tweets batch op errors occurred
file {} processed causar-AND-contagiar-1594293167453.json
83 . -- query None decisión-AND-reproductivo
duplicates inserting decisión-AND-reproductivo tweets 0
error 'KeyError' object has no attribute 'details'
84 . -- query None diu-AND-iniciar
duplicates inserting diu-AND-iniciar tweets 0
error 'KeyError' object has no attribute 'details'
85 . -- query None diu-AND-recomendar
duplicates inserting diu-AND-recomendar tweets batch op errors occurred
file {} processed diu-AND-recomendar-1594278081576.json
86 . -- query None hormona-AND-implantar
duplicates ins

duplicates inserting normal-AND-orgasmo tweets batch op errors occurred
file {} processed normal-AND-orgasmo-1593166346837.json
124 . -- query None hormonal-AND-pastilla
duplicates inserting hormonal-AND-pastilla tweets batch op errors occurred
file {} processed hormonal-AND-pastilla-1594283745204.json
125 . -- query None sexo-AND-sexual
duplicates inserting sexo-AND-sexual tweets batch op errors occurred
file {} processed sexo-AND-sexual-1593138778298.json
126 . -- query None vaginal-AND-doloroso
duplicates inserting vaginal-AND-doloroso tweets batch op errors occurred
file {} processed vaginal-AND-doloroso-1593168990542.json
127 . -- query None anticonceptivo-AND-menstrual
duplicates inserting anticonceptivo-AND-menstrual tweets batch op errors occurred
file {} processed anticonceptivo-AND-menstrual-1594279474933.json
128 . -- query None ano-AND-papiloma
duplicates inserting ano-AND-papiloma tweets batch op errors occurred
file {} processed ano-AND-papiloma-1593128644420.json
129 . -

duplicates inserting pastilla-AND-menstruacion tweets batch op errors occurred
file {} processed pastilla-AND-menstruacion-1593182574173.json
170 . -- query None penar-AND-dolor
duplicates inserting penar-AND-dolor tweets batch op errors occurred
file {} processed penar-AND-dolor-1593145404405.json
171 . -- query {'_id': ObjectId('5ef6380f4060b4e3c8f2d69b'), 'name': 'relacion-AND-resultar', 'lastUpserted': 1593132012257, 'q_versions': [2], 'query': ['relacion', 'resultar'], 'status': 1, 'taxonomy': 'MAA'} relacion-AND-resultar
duplicates inserting relacion-AND-resultar tweets batch op errors occurred
file {} processed relacion-AND-resultar-1594287863973.json
172 . -- query None genital-AND-femenino
duplicates inserting genital-AND-femenino tweets batch op errors occurred
file {} processed genital-AND-femenino-1593151337170.json
173 . -- query None genital-AND-dolor
duplicates inserting genital-AND-dolor tweets batch op errors occurred
file {} processed genital-AND-dolor-1593152000147.j

duplicates inserting relacion-AND-presion tweets batch op errors occurred
file {} processed relacion-AND-presion-1593131865673.json
207 . -- query None página-AND-infectar
duplicates inserting página-AND-infectar tweets 0
error 'KeyError' object has no attribute 'details'
208 . -- query None genital-AND-inflamacion
duplicates inserting genital-AND-inflamacion tweets batch op errors occurred
file {} processed genital-AND-inflamacion-1593151543467.json
209 . -- query None presentar-AND-testiculo
duplicates inserting presentar-AND-testiculo tweets batch op errors occurred
file {} processed presentar-AND-testiculo-1593137043462.json
210 . -- query None vaginal-AND-ereccion
duplicates inserting vaginal-AND-ereccion tweets batch op errors occurred
file {} processed vaginal-AND-ereccion-1593168544622.json
211 . -- query {'_id': ObjectId('5f0756414060b4e3c8f33690'), 'name': 'sangrar-AND-utero', 'lastUpserted': 1594291470887, 'q_versions': [3], 'query': ['sangrar', 'utero'], 'status': 1, 'taxon

error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Network is unreachable
error 54.90.49.105:27017: [Errno 51] Net

392 . -- query None oral-AND-eyacular
duplicates inserting oral-AND-eyacular tweets batch op errors occurred
file {} processed oral-AND-eyacular-1593184684373.json
393 . -- query None ano-AND-madre
duplicates inserting ano-AND-madre tweets batch op errors occurred
file {} processed ano-AND-madre-1593128431258.json
394 . -- query None informacion-AND-trasmision
duplicates inserting informacion-AND-trasmision tweets batch op errors occurred
file {} processed informacion-AND-trasmision-1593135321881.json
395 . -- query None disminuir-AND-secundario
duplicates inserting disminuir-AND-secundario tweets batch op errors occurred
file {} processed disminuir-AND-secundario-1594291893454.json
396 . -- query None vaginal-AND-penetracion
duplicates inserting vaginal-AND-penetracion tweets batch op errors occurred
file {} processed vaginal-AND-penetracion-1593167982565.json
397 . -- query None asesoria-AND-salud
duplicates inserting asesoria-AND-salud tweets batch op errors occurred
file {} process

In [73]:
"keyValue" in error_details[0] and "tweet_id" in error_details[0]["keyValue"]

False

In [71]:
error_details[0]["keyValue"]["tweet_id"]

'646055382'

In [78]:
sorted([tw["tweet_id"] for tw in tweets])

['1030182208475156482',
 '1030182208475156482',
 '1033794663742943232',
 '1033794663742943232',
 '1036144117028454400',
 '1036144117028454400',
 '1036635188267233280',
 '1036635188267233280',
 '1037769335153090566',
 '1037769335153090566',
 '1037808742014902272',
 '1037808742014902272',
 '1040079215851450368',
 '1040079215851450368',
 '1040387975887159297',
 '1040387975887159297',
 '1043569411590967297',
 '1043569411590967297',
 '1046140209413410816',
 '1046140209413410816',
 '1048348657832140800',
 '1048348657832140800',
 '1053946620453240833',
 '1053946620453240833',
 '1054064347457163264',
 '1054064347457163264',
 '1055478052736655361',
 '1055478052736655361',
 '1055478999974068224',
 '1055478999974068224',
 '1058279647476178944',
 '1058279647476178944',
 '1058502115209945088',
 '1058502115209945088',
 '1063039430913605633',
 '1063039430913605633',
 '1068677282410172416',
 '1068677282410172416',
 '1075100776417107968',
 '1075100776417107968',
 '1118615120928628736',
 '11186151209286

In [80]:
[tw for tw in tweets if tw["tweet_id"] == '1030182208475156482']

[{'has_media': True,
  'hashtags': [],
  'img_urls': ['https://pbs.twimg.com/media/DkvxQK5X4AE7rUF.jpg'],
  'is_replied': False,
  'is_reply_to': False,
  'likes': 1,
  'links': [],
  'parent_tweet_id': '',
  'replies': 0,
  'reply_to_users': [],
  'retweets': 0,
  'screen_name': 'mothercarevzla',
  'text': '¿Sabes que tipo de ropa debes usar durante el embarazo?\n\n¡Debes usar ropa cómoda que se ajuste al tamaño de tu vientre, nada que cause presión excesiva sobre la zona, prendas que sean frescas y un poco anchas.\n\nFrom @MothercareDOpic.twitter.com/UiwbGKVZr0',
  'timestamp': '2018-08-16T19:59:13',
  'timestamp_epochs': 1534449553,
  'tweet_id': '1030182208475156482',
  'tweet_url': '/mothercarevzla/status/1030182208475156482',
  'user_id': '973545587248455680',
  'username': 'Mothercare Venezuela',
  'video_url': '',
  'queryName': 'ajuste-AND-embarazo',
  'query': ['ajuste', 'embarazo'],
  '_id': ObjectId('5ed536d5ef2f5847b9ff19de')},
 {'has_media': True,
  'hashtags': [],
  'img

In [46]:
[result.inserted_ids for result in results]

[[ObjectId('5ed50964ef2f5847b9ff14ce'),
  ObjectId('5ed50964ef2f5847b9ff14cf'),
  ObjectId('5ed50964ef2f5847b9ff14d0'),
  ObjectId('5ed50964ef2f5847b9ff14d1'),
  ObjectId('5ed50964ef2f5847b9ff14d2'),
  ObjectId('5ed50964ef2f5847b9ff14d3'),
  ObjectId('5ed50964ef2f5847b9ff14d4'),
  ObjectId('5ed50964ef2f5847b9ff14d5'),
  ObjectId('5ed50964ef2f5847b9ff14d6'),
  ObjectId('5ed50964ef2f5847b9ff14d7'),
  ObjectId('5ed50964ef2f5847b9ff14d8'),
  ObjectId('5ed50964ef2f5847b9ff14d9'),
  ObjectId('5ed50964ef2f5847b9ff14da'),
  ObjectId('5ed50964ef2f5847b9ff14db'),
  ObjectId('5ed50964ef2f5847b9ff14dc'),
  ObjectId('5ed50964ef2f5847b9ff14dd'),
  ObjectId('5ed50964ef2f5847b9ff14de'),
  ObjectId('5ed50964ef2f5847b9ff14df'),
  ObjectId('5ed50964ef2f5847b9ff14e0'),
  ObjectId('5ed50964ef2f5847b9ff14e1'),
  ObjectId('5ed50964ef2f5847b9ff14e2'),
  ObjectId('5ed50964ef2f5847b9ff14e3'),
  ObjectId('5ed50964ef2f5847b9ff14e4'),
  ObjectId('5ed50964ef2f5847b9ff14e5'),
  ObjectId('5ed50964ef2f5847b9ff14e6'),


In [54]:
res = queries_collection.find_one({"name":path_name})
res == None

True

{'_id': ObjectId('5ed505f3a97fd6b214f48558'),
 'taxonomy': 'MMA',
 'name': 'embarazo-AND-mes',
 'query': ['embarazo', 'mes'],
 'lastUpserted': 1590997010547.0}

# actualizar tweets sin similarity

In [14]:
tweets_collection.update({"similarity" :  {'$exists': False}})

/Users/davidbautista/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


41879